In [1]:
import numpy as np
import itertools
import torch
from sklearn.model_selection import train_test_split

In [2]:
N = 16
K = 8

In [3]:
s = list(itertools.combinations(range(N), K))

In [4]:
s[:10]

[(0, 1, 2, 3, 4, 5, 6, 7),
 (0, 1, 2, 3, 4, 5, 6, 8),
 (0, 1, 2, 3, 4, 5, 6, 9),
 (0, 1, 2, 3, 4, 5, 6, 10),
 (0, 1, 2, 3, 4, 5, 6, 11),
 (0, 1, 2, 3, 4, 5, 6, 12),
 (0, 1, 2, 3, 4, 5, 6, 13),
 (0, 1, 2, 3, 4, 5, 6, 14),
 (0, 1, 2, 3, 4, 5, 6, 15),
 (0, 1, 2, 3, 4, 5, 7, 8)]

In [5]:
labels = []
for i in s:
  labels.append([1 if j in i else 0 for j in range(N)])

In [6]:
labels[:10]

[[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]]

In [8]:
inputs= []
for i in range(len(s)):
  inputs.append([int(bit) for bit in format(i, '014b')])

In [9]:
inputs[:10]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]]

In [10]:
inputs = np.array(inputs, dtype=np.float32)
labels = np.array(labels, dtype=np.float32)

In [59]:
xtrain, xtest, ytrain, ytest = train_test_split(inputs, labels, test_size=0.7, random_state=42)

In [60]:
print("Training data shape:", xtrain.shape)
print("Testing data shape:", xtest.shape)
print("Training labels shape:", ytrain.shape)
print("Testing labels shape:", ytest.shape)

Training data shape: (3861, 14)
Testing data shape: (9009, 14)
Training labels shape: (3861, 16)
Testing labels shape: (9009, 16)


In [61]:
xtrain = torch.tensor(xtrain)
ytrain = torch.tensor(ytrain)
xtest = torch.tensor(xtest)
ytest = torch.tensor(ytest)

In [62]:
class AutoEncoder(torch.nn.Module):
  def __init__(self, hidden_dim):
    super().__init__()
    self.encoder = torch.nn.Linear(14, hidden_dim)
    self.decoder = torch.nn.Linear(hidden_dim, 16)
    self.sigmoid = torch.nn.Sigmoid()

  def forward(self, x):
        encoded = self.encoder(x)
        pred = self.decoder(encoded)
        return self.sigmoid(pred)

In [63]:
model = AutoEncoder(8)

In [64]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
print(model)

AutoEncoder(
  (encoder): Linear(in_features=14, out_features=8, bias=True)
  (decoder): Linear(in_features=8, out_features=16, bias=True)
  (sigmoid): Sigmoid()
)


In [65]:
epochs = 150*2

for i in range(epochs):
  model.train()
  optimizer.zero_grad()
  out = model(xtrain)
  loss = criterion(out, ytrain)
  binary_out = (out> 0.5).float()
  #correct = (binary_out == ytrain).float().sum()
  #total = ytrain.numel()
  correct = torch.all(binary_out == ytrain, dim=1).float().sum()
  total = ytrain.size(0)
  accuracy = correct / total * 100

  loss.backward()
  optimizer.step()

  if (i + 1) % 10 == 0:
        print(f"Epoch [{i+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}")



Epoch [10/300], Loss: 0.6890, Accuracy: 0.00
Epoch [20/300], Loss: 0.6801, Accuracy: 0.00
Epoch [30/300], Loss: 0.6698, Accuracy: 0.00
Epoch [40/300], Loss: 0.6606, Accuracy: 0.00
Epoch [50/300], Loss: 0.6538, Accuracy: 0.00
Epoch [60/300], Loss: 0.6484, Accuracy: 0.00
Epoch [70/300], Loss: 0.6441, Accuracy: 0.00
Epoch [80/300], Loss: 0.6411, Accuracy: 0.00
Epoch [90/300], Loss: 0.6390, Accuracy: 0.00
Epoch [100/300], Loss: 0.6374, Accuracy: 0.00
Epoch [110/300], Loss: 0.6363, Accuracy: 0.00
Epoch [120/300], Loss: 0.6354, Accuracy: 0.00
Epoch [130/300], Loss: 0.6346, Accuracy: 0.00
Epoch [140/300], Loss: 0.6341, Accuracy: 0.00
Epoch [150/300], Loss: 0.6336, Accuracy: 0.00
Epoch [160/300], Loss: 0.6332, Accuracy: 0.00
Epoch [170/300], Loss: 0.6329, Accuracy: 0.03
Epoch [180/300], Loss: 0.6326, Accuracy: 0.00
Epoch [190/300], Loss: 0.6324, Accuracy: 0.00
Epoch [200/300], Loss: 0.6321, Accuracy: 0.00
Epoch [210/300], Loss: 0.6320, Accuracy: 0.00
Epoch [220/300], Loss: 0.6318, Accuracy: 0.

In [66]:
class AutoEncoder_v2(torch.nn.Module):
  def __init__(self, hidden_dim):
    super().__init__()
    self.encoder = torch.nn.Sequential(
        torch.nn.Linear(14, hidden_dim),
        torch.nn.Linear(hidden_dim, hidden_dim//2)
    )
    self.decoder = torch.nn.Sequential(
        torch.nn.Linear(hidden_dim//2, hidden_dim),
        torch.nn.Linear(hidden_dim, 16),
        torch.nn.Sigmoid()
    )

  def forward(self, x):
    out = self.encoder(x)
    pred = self.decoder(out)
    return pred

In [67]:
model1 = AutoEncoder_v2(8)

In [68]:
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [69]:
print(model1)

AutoEncoder_v2(
  (encoder): Sequential(
    (0): Linear(in_features=14, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=4, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=16, bias=True)
    (2): Sigmoid()
  )
)


In [70]:
epochs = 150*2

for i in range(epochs):
  model1.train()
  optimizer.zero_grad()
  out = model1(xtrain)
  loss = criterion(out, ytrain)
  binary_out = (out> 0.5).float()
  #correct = (binary_out == ytrain).float().sum()
  #total = ytrain.numel()
  correct = torch.all(binary_out == ytrain, dim=1).float().sum()
  total = ytrain.size(0)
  accuracy = correct / total * 100

  loss.backward()
  optimizer.step()

  if (i + 1) % 10 == 0:
        print(f"Epoch [{i+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}")



Epoch [10/300], Loss: 0.6991, Accuracy: 0.00
Epoch [20/300], Loss: 0.6991, Accuracy: 0.00
Epoch [30/300], Loss: 0.6991, Accuracy: 0.00
Epoch [40/300], Loss: 0.6991, Accuracy: 0.00
Epoch [50/300], Loss: 0.6991, Accuracy: 0.00
Epoch [60/300], Loss: 0.6991, Accuracy: 0.00
Epoch [70/300], Loss: 0.6991, Accuracy: 0.00
Epoch [80/300], Loss: 0.6991, Accuracy: 0.00
Epoch [90/300], Loss: 0.6991, Accuracy: 0.00
Epoch [100/300], Loss: 0.6991, Accuracy: 0.00
Epoch [110/300], Loss: 0.6991, Accuracy: 0.00
Epoch [120/300], Loss: 0.6991, Accuracy: 0.00
Epoch [130/300], Loss: 0.6991, Accuracy: 0.00
Epoch [140/300], Loss: 0.6991, Accuracy: 0.00
Epoch [150/300], Loss: 0.6991, Accuracy: 0.00
Epoch [160/300], Loss: 0.6991, Accuracy: 0.00
Epoch [170/300], Loss: 0.6991, Accuracy: 0.00
Epoch [180/300], Loss: 0.6991, Accuracy: 0.00
Epoch [190/300], Loss: 0.6991, Accuracy: 0.00
Epoch [200/300], Loss: 0.6991, Accuracy: 0.00
Epoch [210/300], Loss: 0.6991, Accuracy: 0.00
Epoch [220/300], Loss: 0.6991, Accuracy: 0.

In [76]:
xtrain, xtest, ytrain, ytest = train_test_split(inputs, labels, test_size=0.7, random_state=42)
print("Training data shape:", xtrain.shape)
print("Testing data shape:", xtest.shape)
print("Training labels shape:", ytrain.shape)
print("Testing labels shape:", ytest.shape)


xtrain = torch.tensor(xtrain)
ytrain = torch.tensor(ytrain)
xtest = torch.tensor(xtest)
ytest = torch.tensor(ytest)

Training data shape: (3861, 14)
Testing data shape: (9009, 14)
Training labels shape: (3861, 16)
Testing labels shape: (9009, 16)


In [77]:

class EncoderDecoder(torch.nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(14, hidden_dim),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim*2),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hidden_dim*2, hidden_dim*2),
            torch.nn.Tanh(),
            #torch.nn.Linear(hidden_dim, hidden_dim*2),
            torch.nn.Dropout(inplace=False)
            #torch.nn.
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim*2, hidden_dim*2),
            torch.nn.LeakyReLU(),
            #torch.nn.Tanh(),
            torch.nn.Linear(hidden_dim*2, hidden_dim),
            torch.nn.LeakyReLU(),
            #torch.nn.Dropout(inplace=True),
            torch.nn.Linear(hidden_dim, 16),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        pred = self.decoder(encoded)
        return pred

model = EncoderDecoder(256)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
print(model)

EncoderDecoder(
  (encoder): Sequential(
    (0): Linear(in_features=14, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Tanh()
    (6): Dropout(p=0.5, inplace=False)
  )
  (decoder): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=256, out_features=16, bias=True)
    (5): Sigmoid()
  )
)


In [78]:

epochs = 1000
for i in range(epochs):
  model.train()
  optimizer.zero_grad()
  out = model(xtrain)
  loss = criterion(out, ytrain)
  binary_out = (out> 0.5).float()
  #correct = (binary_out == ytrain).float().sum()
  #total = ytrain.numel()
  correct = torch.all(binary_out == ytrain, dim=1).float().sum()
  total = ytrain.size(0)
  accuracy = correct / total * 100

  loss.backward()
  optimizer.step()

  if (i + 1) % 10 == 0:
        print(f"Epoch [{i+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}")

model.eval()
with torch.no_grad():
  out = model(xtest)
  binary_out_test = (out > 0.5).float()

  #correct = (binary_out_test == ytest).float().sum()
  #total = ytest.numel()
  correct = torch.all(binary_out_test == ytest, dim=1).float().sum()
  total = ytest.size(0)
  accuracy = correct / total * 100

  print(f"Test Accuracy: {accuracy:.2f}%")

Epoch [10/1000], Loss: 0.6934, Accuracy: 0.00
Epoch [20/1000], Loss: 0.6787, Accuracy: 0.00
Epoch [30/1000], Loss: 0.6478, Accuracy: 0.00
Epoch [40/1000], Loss: 0.6262, Accuracy: 0.00
Epoch [50/1000], Loss: 0.6023, Accuracy: 0.03
Epoch [60/1000], Loss: 0.5793, Accuracy: 0.00
Epoch [70/1000], Loss: 0.5622, Accuracy: 0.03
Epoch [80/1000], Loss: 0.5393, Accuracy: 0.03
Epoch [90/1000], Loss: 0.9617, Accuracy: 0.05
Epoch [100/1000], Loss: 0.6122, Accuracy: 0.00
Epoch [110/1000], Loss: 0.5540, Accuracy: 0.03
Epoch [120/1000], Loss: 0.5131, Accuracy: 0.05
Epoch [130/1000], Loss: 0.4861, Accuracy: 0.03
Epoch [140/1000], Loss: 0.4652, Accuracy: 0.00
Epoch [150/1000], Loss: 0.4464, Accuracy: 0.13
Epoch [160/1000], Loss: 0.4276, Accuracy: 0.05
Epoch [170/1000], Loss: 0.4141, Accuracy: 0.28
Epoch [180/1000], Loss: 0.4039, Accuracy: 0.41
Epoch [190/1000], Loss: 0.3877, Accuracy: 0.36
Epoch [200/1000], Loss: 0.3868, Accuracy: 0.62
Epoch [210/1000], Loss: 0.3682, Accuracy: 0.70
Epoch [220/1000], Loss

In [ ]:
import torch
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
N = 16
K = 8
subsets = list(itertools.combinations(range(N), K))
labels = []
for i in subsets:
  labels.append([1 if num in i else 0 for num in range(N)])
inputs = []
for i in range(len(subsets)):
   inputs.append([int(bit) for bit in format(i, '014b')])

inputs = np.array(inputs, dtype=np.float32)
labels = np.array(labels, dtype=np.float32)

xtrain, xtest, ytrain, ytest = train_test_split(inputs, labels, test_size=0.7, random_state=42)
print("Training data shape:", xtrain.shape)
print("Testing data shape:", xtest.shape)
print("Training labels shape:", ytrain.shape)
print("Testing labels shape:", ytest.shape)


xtrain = torch.tensor(xtrain)
ytrain = torch.tensor(ytrain)
xtest = torch.tensor(xtest)
ytest = torch.tensor(ytest)

class EncoderDecoder(torch.nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(14, hidden_dim),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim*2),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(hidden_dim*2, hidden_dim*2),
            torch.nn.Tanh(),
            #torch.nn.Linear(hidden_dim, hidden_dim*2),
            torch.nn.Dropout(inplace=False)
            #torch.nn.
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim*2, hidden_dim*2),
            torch.nn.LeakyReLU(),
            #torch.nn.Tanh(),
            torch.nn.Linear(hidden_dim*2, hidden_dim),
            torch.nn.LeakyReLU(),
            #torch.nn.Dropout(inplace=True),
            torch.nn.Linear(hidden_dim, 16),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        pred = self.decoder(encoded)
        return pred

model = EncoderDecoder(256)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
print(model)

epochs = 1000
for i in range(epochs):
  model.train()
  optimizer.zero_grad()
  out = model(xtrain)
  loss = criterion(out, ytrain)
  binary_out = (out> 0.5).float()
  #correct = (binary_out == ytrain).float().sum()
  #total = ytrain.numel()
  correct = torch.all(binary_out == ytrain, dim=1).float().sum()
  total = ytrain.size(0)
  accuracy = correct / total * 100

  loss.backward()
  optimizer.step()

  if (i + 1) % 10 == 0:
        print(f"Epoch [{i+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}")

model.eval()
with torch.no_grad():
  out = model(xtest)
  binary_out_test = (out > 0.5).float()

  #correct = (binary_out_test == ytest).float().sum()
  #total = ytest.numel()
  correct = torch.all(binary_out_test == ytest, dim=1).float().sum()
  total = ytest.size(0)
  accuracy = correct / total * 100

  print(f"Test Accuracy: {accuracy:.2f}%")
